# Example 143: EPU Power Density - Multi-particle

In this example we calculate the multi-particle power density on a rectangular surface 30 [m] downstream from the center of the EPU.

In [ ]:
# This has nothing to do with OSCARS, but it puts the matplotlib plots inline in the notebook
%matplotlib inline

# Import the OSCARS SR module
import oscars.sr

# Import basic plot utilities (matplotlib).  You don't need these to run OSCARS, but it's used here for basic plots
from oscars.plots_mpl import *

In [ ]:
# Create a new OSCARS object.  Default to 8 threads and always use the GPU if available
osr = oscars.sr.sr(nthreads=8, gpu=1)

## Create the undulator field

Here we create an EPU by adding two ideal undulators, one with a vertical field and one with a horizontal field, out of phase.

For the undulator, *bfield* represents maximum magnetic field [$B_x, B_y, B_z$].  The *period* is also in vector form which allows you to orient the axis of the undulator in any arbitrary direction.  The number of periods is given by *nperiods*.  This is the number of FULL periods.  A terminating field of 1 period length is added to each side in addition to *nperiods*.

Typically clear_magnetic_fields() is called before adding a field in notebooks only to save time when making changes and rerunning sections of the code so it is not strictly necessary.

In [ ]:
# Phase difference between fields in [rad]
phase = osr.pi()/2.

# Clear any existing fields (just good habit in notebook style) and add an undulator field
osr.clear_bfields()
osr.add_bfield_undulator(bfield=[0, 0.7, 0], period=[0, 0, 0.049], nperiods=21, phase=-phase/2.)
osr.add_bfield_undulator(bfield=[0.7, 0, 0], period=[0, 0, 0.049], nperiods=21, phase=+phase/2.)

# Just to check the field that we added seems visually correct
plot_bfield(osr)

## Add a particle beam

Now we add a simple particle beam.  There are a few important things to understand about the setup of magnetic fields and particle beams, namely positions and times.  You can define the initial position and time of a particle beam to be anywhere you like.  The default initial time for a beam is t=0.  You must specify the initial position, which for this simple example it is recommended to be *before* the undulator (in the plot at z=-1 [m] is a good choice if the particle beam is defined to be going in the +z direction as is done below.

One must specify ctstartstop.  This is the start and stop time of the calculation.  In this example we will start the calculation at t=0 and go to t=2 (given in units of ct) since the beam is relativistic.  In this example you can specify the start time as less than 0 which is useful if you want to propogate the particle backwars in time.  This is useful for instance if you have a bending magnet before the undulator that you wish to include.

clear_particle_beams() is called, again for convenience, but it is not necessary.

In [ ]:
# Setup beam similar to NSLSII
osr.clear_particle_beams()
osr.set_particle_beam(type='electron',
                      energy_GeV=3,
                      x0=[0, 0, -1],
                      current=0.500,
                      sigma_energy_GeV=0.001*3,
                      beta=[1.5, 0.8],
                      emittance=[0.9e-9, 0.008e-9]
                     )

# Set the start and stop times for the calculation
osr.set_ctstartstop(0, 2)

## Calculate Trajectory

Now we calculate the trajectory and plot it.  It is enough to call calculate_trajectory().  If you are doing other calculations (flux, spectra, power density) it is not necesary to call this since it is called internally.

In [ ]:
# Run the particle trajectory calculation for the ideal particle
# First must load the ideal particle initial conditions
osr.set_new_particle(particle='ideal')
trajectory = osr.calculate_trajectory()

# Plot the trajectory position and velocity
plot_trajectory_position(trajectory)
plot_trajectory_velocity(trajectory)

## Calculate Power Density

In [ ]:
# Calculate the multi-particle Flux.  We'll pick a point around 154 [eV]
# Although 5 particles are used here, you shuold use a lot more.
pd = osr.calculate_power_density_rectangle(plane='XY',
                                           width=[0.05, 0.05],
                                           npoints=[101, 101],
                                           translation=[0, 0, 30],
                                           nparticles=5)
plot_power_density(pd)